In [ ]:
#clone using 'git clone https://github.com/GoogleCloudPlatform/training-data-analyst '
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-00-79c1e4f2f7e0'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
##Creating GCP bucket
# %%bash
# if ! gsutil ls | grep -q gs://${BUCKET}/; then
#   gsutil mb -l ${REGION} gs://${BUCKET}
# fi

In [ ]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

In [ ]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

In [ ]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
##Train = 75%
trainQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE MOD(hashmonth, 4) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

In [ ]:
traindf.head()

In [ ]:
# Let's look at a small sample of the training data
traindf.describe()

In [ ]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [ ]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

In [ ]:
traindf.tail()

In [ ]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

In [1]:
#In the final versions, we want to read from files, not Pandas dataframes. 
#So, write the Pandas dataframes out as CSV files. Using CSV files gives us the advantage
#of shuffling during read. This is important for distributed training because some workers 
#might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers. 

traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

SyntaxError: invalid syntax (<ipython-input-1-9619bf2943f6>, line 1)

In [ ]:
%%bash
wc -l *.csv
head *.csv
tail *.csv